In [ ]:
# UNCOMMENT IF ON REMOTE JUPYTER
# !pip uninstall pmdarima statsmodels matplotlib --yes
# !pip install protobuf==3.20.0 tensorflow==2.6.2 plotly pandas optuna

In [ ]:
import logging
import sys, os
import optuna
import pandas as pd
import tensorflow as tf
from sklearn.metrics import mean_absolute_error
from tensorflow.keras import Input, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Bidirectional
from tensorflow.keras.optimizers import Nadam
from sklearn.compose import ColumnTransformer
from keras.layers import LeakyReLU
from data_util_common import canopy_dataset, train_val_test_split
from lstm_util import prepare_data, evaluate

In [ ]:
tf.random.set_seed(42)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
targets = logging.StreamHandler(sys.stdout), logging.FileHandler('lstm.log')
logging.basicConfig(format='%(message)s', level=logging.INFO, handlers=targets)
format = "%Y-%m-%d"

In [ ]:
# DEFINE OPTUNA TRIAL
def objective(trial):   
    models = dict()
    models[1] = [168, 48, 24]
    models[2] = [168, 72, 48, 24]
    models[3] = [24, 24, 24]
      
    idx = trial.suggest_categorical("model", [x for x in range(1, len(models))])
    bidirectional = trial.suggest_categorical("bidirectional", [True, False])
    dense_activation = trial.suggest_categorical("dense_act", ['relu', 'leaky relu', 'linear', 'tanh'])

    for batch in dataset_train.take(1):
        inputs, targets = batch

    model = Sequential()
    model.add(Input((inputs.shape[1], inputs.shape[2]), None))
    for unit in models[idx]:
       if bidirectional:
            model.add(Bidirectional(LSTM(unit, return_sequences=True)))
       else:
          model.add(LSTM(unit, return_sequences=True))

    if dense_activation == 'leaky relu':
        model.add(TimeDistributed(Dense(1)))
        model.add(LeakyReLU())
    else:
       model.add(TimeDistributed(Dense(1, activation=dense_activation)))
    
    model.compile(optimizer=Nadam(learning_rate=0.0001), loss="mae")

    checkpoint_filepath = f'optimization/checkpoint_{trial.number}.h5'
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, 
                        monitor="val_loss",
                        save_best_only=True, verbose=0)
    
    history = model.fit(
        dataset_train,
        epochs=1000,
        validation_data=dataset_val,
        shuffle=False,
        callbacks=[EarlyStopping(monitor="val_loss", patience=30, min_delta=0.01), checkpoint],
        verbose=0
    )
     
    model.load_weights(checkpoint_filepath)
   
    days = int((test_size-past)/T)
    results = evaluate(dataset_test, model, days, pipeline.named_transformers_['label'])        
    mae = mean_absolute_error(results['y_true'], results['y_pred'])
    return mae


In [ ]:
data, label = canopy_dataset()
data = data[[label, 
            'Irradiance_1_Wm2'
            ]]

print(data.columns)
label_idx = list(data.columns).index(label)
print(label_idx)

T = 24
past = T * 7 
future = T
step = 1
batch_size = 512
sequence_length = int(past / step)

print(f"RESOLUTION: {T} - LOOKBACK WINDOW SIZE: {past} - PREDICTION WINDOW SIZE: {future}")

# SPLIT DATA IN TRAIN AND TEST
split_idx = train_val_test_split(pd.to_datetime(data.index), [80, 20])
train_val_size = split_idx[0]
train_val_split = train_val_test_split(data.index[:train_val_size], [70, 30])
train_size = train_val_split[0]
val_size = train_val_size - train_size

train_val_size = split_idx[0]
train = data.iloc[:train_val_size, :]

# scale data
numeric_features=['Irradiance_1_Wm2']

features_scaler = MinMaxScaler()
label_scaler = MinMaxScaler()

pipeline=ColumnTransformer([
    ('label', MinMaxScaler(), [label]),
    ('num', MinMaxScaler(), numeric_features),
], remainder='passthrough')

# SPLIT DATA IN TRAIN AND TEST
split_idx = train_val_test_split(pd.to_datetime(data.index), [80, 20])
train_val_size = split_idx[0]
train = data.iloc[:train_val_size, :]

train=pipeline.fit_transform(train)

# SPLIT TRAIN IN SUB-TRAIN AND VALIDATION
train_val_split = train_val_test_split(data.index[:train_val_size], [70, 30])
train_size = train_val_split[0]
val_size = train_val_size - train_size

val = train[train_size:, :]
train = train[:train_size, :]

# GET TEST VALUES
test = data.iloc[train_val_size:, :]
test = pipeline.transform(test)
test_size = len(test)
test_timestamp = data.index[train_val_size:]
print(f"TRAIN {train_size} VAL {val_size} TEST {test_size} - TOTAL {len(data)}")

#####################################################################################################################
# PREPARE DATA
# TRAIN
dataset_train = prepare_data(train, 0, train_size-past, past, train_size, sequence_length, 1, batch_size, label_idx)

# VAL
dataset_val = prepare_data(val, 0, val_size-past, past, val_size, sequence_length, 1, batch_size, label_idx)

# TEST
dataset_test = prepare_data(test, 0, test_size, past, test_size, sequence_length, 1, batch_size, label_idx)

print("*** TRAINING *** --------------------------------------------------------------------------------------")
for batch in dataset_train.take(1):
    inputs, targets = batch

print(f"Input shape:  {inputs.numpy().shape}")
print(f"Target shape: {targets.numpy().shape}")

In [ ]:
#OPTUNA TRIALS
with tf.device('/gpu:0'):
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_jobs=2, show_progress_bar=True, n_trials=50)

with open("optimization_Res.txt", "w") as f:
    f.write("Number of finished trials: {}".format(len(study.trials)))
    f.write("Best trial:")
    
    trial = study.best_trial
    f.write("  Value: {}".format(trial.value))
    
    f.write("  Params: ")
    for key, value in trial.params.items():
        f.write("    {}: {}".format(key, value))